In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
hfscatfilesraw = os.listdir('catalog_hfs')
hfscatfilesraw.sort()
hfscatfiles = [x for x in hfscatfilesraw if "backup" not in x]
hfscatcodes = []
for hfscatfile in hfscatfiles:
    hfscatcodes.append(hfscatfile.split("_")[0].replace("c00","").replace("c0","").replace("c",""))
len(hfscatcodes)

In [ ]:
hfsdict = dict(zip(hfscatcodes,hfscatfiles))
hfsdict

In [ ]:
# CHECK REPEATED
seen = set()
uniq = []
print("REPEATED ENTRIES")
for x in hfscatcodes:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
    else:
        print(x)
        matching = [s for s in hfscatfiles if x in s]
        print (matching)        

In [ ]:
fpartition = open('partition_function.html',"r")
fpartitionlines = fpartition.readlines()
fpartition.close()

In [ ]:
# Comprobacion de que la suma de las intensidades en el catalogo para el no hfs es igual a la suma de todas las hfs de la misma transicion.
def calculateCorrectionFactor(fpartitioncode):
    # Tolerancia en frecuencia para asumir que es la misma linea
    threshold=0.02
    # Leo el .cat de la nohfs y la hfs en un pandas y solo tomo las columnas de frequency y I3000
    molec = pd.read_fwf('catalog/%s' % hfsdict[fpartitioncode].replace('_hfs',''),header = None,colspecs=[(0,13),(21,29)])
    molechfs = pd.read_fwf('catalog_hfs/%s' % hfsdict[fpartitioncode],header = None,colspecs=[(0,13),(21,29)])
    # Tomo la segunda frecuencia del catalogo (por no pillar la hiperfina de un borde)
    reffreq = molechfs.iloc[1][0] # Not the first hiperfine line
    # muestro la frecuencia de esa transicion
    print ("Representative frequency: %s" %reffreq)
    # Sumo todas las 10^I300 de todas las transiciones que esten a +-thresholde de esa frecuencia, identificandolas como hfs
    moleclineshfs = np.array(molechfs[(molechfs[0]>reffreq*(1-threshold)) & (molechfs[0]<reffreq*(1+threshold))][1])
    moleclineshfssum = np.power(10,moleclineshfs).sum()
    # Muestro la suma
    print ("Sum HFS: %s" %moleclineshfssum)
    # Lo mismo para la no hfs
    moleclines = np.array(molec[(molec[0]>reffreq*(1-threshold)) & (molec[0]<reffreq*(1+threshold))][1])
    moleclinessum = np.power(10,moleclines).sum()
    print ("Sum : %s" %moleclinessum)
    # Muestro el ratio. Si todo esta ok, esto deberia ser aproximadamente 1
    print ("MOLEC/MOLECHFS: %s "%(moleclinessum/moleclineshfssum))
    print("----------FACTOR: %s" %int(round(moleclinessum/moleclineshfssum)))
    return int(round(moleclinessum/moleclineshfssum))

In [ ]:
fpartitionoutput = open('partition_function_hfs.html',"w")
hfsfoundcodes = []
for fpartitionline in fpartitionlines:
    fpartitioncode = fpartitionline[0:6].replace(" ","")
    if fpartitioncode in hfscatcodes:
        hfsfoundcodes.append(fpartitioncode)
        print("%s---------%s" %(fpartitionline[7:33],fpartitioncode))
        #factor = calculateCorrectionFactor(fpartitioncode)
        auxmolname =fpartitionline[7:33].rstrip()
        #print(fpartitionline[7:33].replace("%s    "%auxmolname,"%s,hfs"%auxmolname)+"---")
        fpartitionoutput.write(fpartitionline.replace("%s    "%auxmolname,"%s,hfs"%auxmolname))
fpartitionoutput.close()

print("")
print("FOUND %s out of %s" %(len(hfsfoundcodes), len(hfscatcodes)))
print("NOT FOUND CODES (consider repeated above)")
list (set(hfscatcodes)-set(hfsfoundcodes))

In [ ]:
calculateCorrectionFactor('28501')

In [ ]:
hfscatcodes